## Apiux & SII: Analisis exploratorio de datos y depuracion de data sociedades
## ATENCION: proyecto sujeto a mantenimiento continuo. 

## Henry Vega (henrry.vega@api-ux.com)
## Data analyst

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
#warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
ss_name = 'Depuracion malla societaria'
wg_conn = "spark.kerberos.access.hadoopFileSystems"
db_conn = "abfs://data@datalakesii.dfs.core.windows.net/"

spark = SparkSession.builder \
      .appName(f"Ejecucion algoritmo {ss_name}")  \
      .config(wg_conn, db_conn) \
      .config("spark.executor.memory", "6g") \
      .config("spark.driver.memory", "12g")\
      .config("spark.executor.cores", "4") \
      .config("spark.executor.instances", "5") \
      .config("spark.driver.maxResultSize", "12g") \
      .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")



Setting spark.hadoop.yarn.resourcemanager.principal to hvega.externo


## Carga de relaciones societarias y depuracion de data

Primero, veamos los valores null en participacion de capital y participacion de utilidades.

In [3]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES from libsdf.jab_soc_2023_inom where PORCENTAJE_CAPITAL is null or PORCENTAJE_UTILIDADES IS NULL").show()

Hive Session ID = 4b4dac8c-9c5c-45e9-9016-e3fa29ce9f3d


+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+



Vemos que no hay valores nulos en la tabla. A continuacion veamos cuantos duplicados existen en las columnas de interes:
RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES, pues todos
los calculos lo haremos basados en esta columnas.

In [4]:
spark.sql("select * from libsdf.jba_soc_2022_e where RUT_SOCIEDAD like 'dwElNqcCQQiFyI3ic%' ").show()

+------------+---------+------------------+---------------------+------------------------+------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|PERI_AGNO_MES_TRIBUTARIO|FUENTE|
+------------+---------+------------------+---------------------+------------------------+------+
+------------+---------+------------------+---------------------+------------------------+------+



In [5]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES, count(*) as c from libsdf.jba_soc_2023_e  group by  RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES order by c desc").createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad").show()
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES from sociedad").createOrReplaceTempView("sociedad")

+--------------------+--------------------+------------------+---------------------+---+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|  c|
+--------------------+--------------------+------------------+---------------------+---+
|lvhKR91n05T3mstWg...|tnshXz2zZHhF/35iM...|             100.0|                100.0| 33|
|FJ59A3rCUrWEuEI/g...|AtxwrhZVZnI1sYNt/...|              50.0|                 50.0| 15|
|U9dRqzVEukCZdQmX8...|qlm4skAGTxB/gAmri...|             100.0|                100.0| 14|
|gwbsyFNCMsaytOcg8...|NvQrSPRmMfK9N1riN...|               0.0|                  0.0| 13|
|jivM6l8xeUjKSU5fI...|26fBX92ujXPn2atcj...|             100.0|                100.0| 12|
|g/CiBABMUw9aF9wzh...|HKzOgOZxwB4u6FlkN...|             100.0|                100.0| 12|
|YERESpwuZelDrFKfk...|aefg+HqgBVthzV/Dp...|              20.0|                 20.0| 12|
|51GDT15LxNxXvHK+5...|rhswzqGrtBdFr5AJW...|              60.0|                 60.0| 12|
|hEQr1w9LgGXEeDym5...

Donde seleccionamos los valores no repetidos. Haciendo nuevamente un recuento de los valores unicos,

In [6]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, count(*) as count from sociedad group by RUT_SOCIEDAD, RUT_SOCIO order by count desc ").show()

+--------------------+--------------------+-----+
|        RUT_SOCIEDAD|           RUT_SOCIO|count|
+--------------------+--------------------+-----+
|r4Sbvds9bDfjFW7e7...|oLYrIqi8xKeaZ0d8x...|    1|
|gwbsyFNCMsaytOcg8...|JktJAv15GbkeHsV7z...|    1|
|MXwvKCmRE3ePFZEwx...|Tb7TtRs4TbmWcxKbL...|    1|
|gwbsyFNCMsaytOcg8...|wCTNU9G2VdOqGWpCk...|    1|
|b1A2ITXUJDff/zZwz...|mJUiswn7ja+8me3YU...|    1|
|McezftPM/3B6l6VWo...|7crTL6ussUkGsc9DC...|    1|
|gwbsyFNCMsaytOcg8...|1+E4iSamBVKgkTuQ0...|    1|
|gwbsyFNCMsaytOcg8...|K1+PJpBALoLNrPLtv...|    1|
|0uABvk9//PTVao/jD...|sg1y/kOt6GOKTc8Pf...|    1|
|Y9drG2styXtkT3RoY...|EvZ1VIB3xbh/0hfUD...|    1|
|MczPsyIcy9kgeID+p...|TNtGzg85S+cL7Frqv...|    1|
|gwbsyFNCMsaytOcg8...|ZKcSHadGgZVXL9VYx...|    1|
|bl5KbSeQnZAHPojzT...|BPHajYEYW0aglXhcm...|    1|
|ScUcIfB8ZWKTSMz3H...|+ijl7Dp0qMCEZEr76...|    1|
|bE/3BaIKhTMvcPTA1...|YChlyWqjkA251d1XH...|    1|
|u+UPhEHbKoRicYxRI...|uQZ0BdNcVx1aQu8do...|    1|
|SGvqyBD202jsXY6Na...|k+d++STtlTZhtRtHn...|    1|


Por lo visto tenemos unicidad de la relaciones sociedad socio, a juzgar por el recuento de las combinaciones. Ahora veremos cuanto suman los valores de PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES para cada una de las sociedades.


In [7]:
spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD order by CAPITAL DESC").show()

+--------------------+-------+----------+
|        RUT_SOCIEDAD|CAPITAL|UTILIDADES|
+--------------------+-------+----------+
|W3DndtzyfOsAQy8O3...|  110.0|     100.0|
|G+yS31d6iCTqeuggN...|  110.0|     110.0|
|VRNdJkUTYXLWMRkSM...|  110.0|     110.0|
|f1zMRTsAknUF9weV0...|  110.0|     110.0|
|+P/D3dLQdBXBfBBMk...|  110.0|     110.0|
|n5MD+Ght8Cx8X+xhK...|  110.0|     110.0|
|KmKCw9Hb8gOrM1Z9W...|  110.0|     100.0|
|ZWuslkhcOMdiyrD/W...|  110.0|     110.0|
|EpHOFngzeSBjKqble...|  110.0|     110.0|
|tMloWBszd+pnK6pjz...|  110.0|     110.0|
|HmjHOVQ8qk9aGu30J...|  110.0|     110.0|
|MSvsNkqEo2nh7yGTH...|  110.0|     110.0|
|CP3xFFso/QqV9VaqJ...|  110.0|     110.0|
|yBR0RNSMY5VYroBj3...|  110.0|     110.0|
|frSs4/GZOUTCL9+U+...|  110.0|     110.0|
|s+CDuiI+aTB8rRFTE...|  110.0|     110.0|
|EqLVYjdaxkTliFuxD...|  110.0|     110.0|
|4ycAGBELEjW5Sts2g...|  110.0|     100.0|
|VQTk89mGDDDvTgvgP...|  110.0|     110.0|
|PQS6x5nDlb1V3Kt8s...|  110.0|     100.0|
+--------------------+-------+----

In [8]:
spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD order by CAPITAL ASC").show()

+--------------------+------------------+------------------+
|        RUT_SOCIEDAD|           CAPITAL|        UTILIDADES|
+--------------------+------------------+------------------+
|gwbsyFNCMsaytOcg8...|11.289999999999841|11.289999999999841|
|pXvrYgUFd1fJaX9eK...|              90.0|              90.0|
|Myrjc0ohOODY5E+5I...|              90.0|              90.0|
|71W9xagfcj+UvZDp+...|              90.0|              90.0|
|3kMoAJwGVlUqorW5F...|              90.0|              90.0|
|a7blMtvMd8GLU7/Gf...|              90.0|              90.0|
|EmEOpWiJWgMhvnVUe...|              90.0|              90.0|
|3tZmOWm/3ud0vraJE...|              90.0|              90.0|
|9a2uxeGAbF+6YoEdK...|              90.0|              90.0|
|lLI9g3XQG9m/MTp84...|              90.0|              90.0|
|LzquuE1pFJM8J+RIo...|              90.0|              90.0|
|JebgGcb1DMf+xgRsy...|              90.0|             100.0|
|xW374WH/FHLHBTlre...|              90.0|              90.0|
|eWKG01GX1QZnp6rsL...|  

Ahora vamos las entradas con al menos un valor cero (que indica cero participacion porcentual)

In [9]:
spark.sql("select COUNT(*) from sociedad WHERE PORCENTAJE_CAPITAL=0 OR PORCENTAJE_UTILIDADES=0").show()

+--------+
|count(1)|
+--------+
| 1060913|
+--------+



Por lo visto, tenemos 1057765 entradas donde al menos uno de ambos porcentajes es cero. Por otro lado, tenemos 
1053936 registros donde ambos porcentajes son cero.


Ahora veamos cuales porcentajes de capital son cero y luego los porcentajes de utilidades son cero.

In [10]:
spark.sql("select count(*) from sociedad WHERE PORCENTAJE_CAPITAL=0 and PORCENTAJE_UTILIDADES!=0").show()
spark.sql("select count(*) from sociedad WHERE PORCENTAJE_CAPITAL!=0 and PORCENTAJE_UTILIDADES=0").show()

+--------+
|count(1)|
+--------+
|    2484|
+--------+



+--------+
|count(1)|
+--------+
|    1231|
+--------+



Para el analisis del problema de oscuridad, es mejor tener en cuenta los porcentajes de participacion de capital, porque los creditos se reparten segun la participacion societaria.
Ahora veamos cuantos tienen valores positivos mayores que 100 o negativos.

In [11]:
spark.sql("select * from sociedad WHERE PORCENTAJE_CAPITAL<0 or PORCENTAJE_CAPITAL>100 or PORCENTAJE_UTILIDADES<0 or PORCENTAJE_UTILIDADES>100").show()

+--------------------+--------------------+------------------+---------------------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+--------------------+--------------------+------------------+---------------------+
|7ZfJs1v+u1B0SCmQJ...|K4PMvCs8vXhziffJN...|            100.01|                100.0|
+--------------------+--------------------+------------------+---------------------+



El cual es solo un valor levemente superior a 100 %. Seleccionamos los que no tienen valores cero en PORCENTAJE_CAPITAL.
IMPORTANTE: de ser utilizado PORCENTAJE_UTILIDADES se debe filtrar sobre esa columna.

In [12]:
spark.sql("select * from sociedad where PORCENTAJE_CAPITAL!=0").createOrReplaceTempView("sociedad")

In [13]:
spark.sql("select count(*) from sociedad where RUT_SOCIEDAD LIKE 'Qbau/6SlJ/lEcKUD%'").show()
spark.sql("select *  from libsdf.jba_soc_2022_e where RUT_SOCIEDAD LIKE 'Qbau/6SlJ/lEcKUD%'").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



+------------+---------+------------------+---------------------+------------------------+------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|PERI_AGNO_MES_TRIBUTARIO|FUENTE|
+------------+---------+------------------+---------------------+------------------------+------+
+------------+---------+------------------+---------------------+------------------------+------+



In [14]:

df=spark.sql("select * from sociedad")
df.write.mode('overwrite').format("parquet").save("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/riesgo_fraude/materia_oscura/malla_procesada")
spark.stop()